# 0. Preparação

In [160]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# requirements:

# análise e manipulação de dados
import pandas as pd
import numpy as np

# visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# pré-processamento de dados
from sklearn.preprocessing import StandardScaler

# redução de dimensionalidade
from sklearn.decomposition import PCA

# modelos de ML

from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans, AgglomerativeClustering

# avaliação de modelos
from sklearn.model_selection import StratifiedGroupKFold, cross_val_score
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_auc_score,
    roc_curve, silhouette_score
)
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 1. Pré-processamento

Questão 1) A. Faça o pré-processamento dos dados e deixe-os preparados para aplicar o modelo

In [2]:
import pandas as pd

# carregamento do csv
# caminho = "/content/drive/My Drive/content/prova2/sinais.csv"
caminho = "sinais.csv"
df_csv = pd.read_csv(caminho)

In [3]:
# visualizar quantidade de linha e colunas
df_csv.shape

(2502, 7)

In [4]:
# 5 primeiras linhas
df_csv.head()

,file_name,width,height,duration_sec,num_frames,sinal,interprete
0,Adição_AP_10.json,738,1008,4.533333,136,Adição,Alexson
1,Adição_AP_1.json,774,1006,4.766667,143,Adição,Alexson
2,Adição_AP_2.json,760,1002,4.433333,133,Adição,Alexson
3,Adição_AP_3.json,762,1000,4.933333,148,Adição,Alexson
4,Adição_AP_4.json,764,1004,4.600000,138,Adição,Alexson


## 1.1 Análise inicial

Buscando valores nulos ou duplicados

In [5]:
df_csv.isnull().sum()

file_name       1
width           0
height          0
duration_sec    0
num_frames      0
sinal           0
interprete      0
dtype: int64

Há um arquivo nulo em "file_name"

In [6]:
df_csv.duplicated().sum()

np.int64(0)

Não há valores duplicados

Análise das colunas

In [7]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2502 entries, 0 to 2501
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   file_name     2501 non-null   object 
 1   width         2502 non-null   int64  
 2   height        2502 non-null   int64  
 3   duration_sec  2502 non-null   float64
 4   num_frames    2502 non-null   int64  
 5   sinal         2502 non-null   object 
 6   interprete    2502 non-null   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 137.0+ KB


*   file_name: nome do arquivo
*   width: largura do vídeo
*   height: altura do vídeo
*   duration_sec: duração do vídeo em segundos
*   num_frames: quantidade de frames do vídeo
*   sinal: sinal sendo sinalizado no vídeo
*   interprete: nome da pessoa que aparece no vídeo







O dado faltante na coluna "file_name" que descobri anteriormente vai ser dropado:

In [8]:
df_csv.dropna(inplace=True)

In [9]:
df_csv.isnull().sum() # conferindo do drop

file_name       0
width           0
height          0
duration_sec    0
num_frames      0
sinal           0
interprete      0
dtype: int64

In [10]:
df_csv.sinal.value_counts()

sinal
Adição              101
Aluno               101
Antropologia        100
Apontador           100
Apostila            100
Biologia            100
Capítulo            100
Classe              100
Coerência           100
Ensinar             100
Coesão              100
Colega              100
Conceito            100
Contexto            100
Curso               100
Dicionário          100
Disciplina          100
Física              100
Escola              100
Estudar             100
Filosofia           100
História            100
Geografia           100
Ângulo              100
Bolsa de Estudos     99
Name: count, dtype: int64

A base está bem balanceada. Apenas 2 sinais têm uma instância a mais que os demais, e um tem uma a menos

In [11]:
df_csv.shape

(2501, 7)

Normalização das strings:

In [12]:
import unicodedata

def remove_acentos(texto: str) -> str:
    if isinstance(texto, str):
        # Normaliza para decompor acentos
        nfkd_form = unicodedata.normalize("NFKD", texto)
        # Filtra tudo que não for "Mark, nonspacing" (ou seja, acentos)
        return "".join([c for c in nfkd_form if not unicodedata.category(c) == "Mn"])
    return texto

In [13]:
# normalizando as colunas "file_name" e "interprete"
# não normalizei "sinal" por ser o atributo alvo

df_csv["file_name"] = df_csv["file_name"].apply(remove_acentos)
df_csv["interprete"] = df_csv["interprete"].apply(remove_acentos)

print(df_csv.head())


           file_name  width  height  duration_sec  num_frames   sinal  \
0  Adicao_AP_10.json    738    1008      4.533333         136  Adição   
1   Adicao_AP_1.json    774    1006      4.766667         143  Adição   
2   Adicao_AP_2.json    760    1002      4.433333         133  Adição   
3   Adicao_AP_3.json    762    1000      4.933333         148  Adição   
4   Adicao_AP_4.json    764    1004      4.600000         138  Adição   

  interprete  
0    Alexson  
1    Alexson  
2    Alexson  
3    Alexson  
4    Alexson  


## Análise dos dados com Profiling

In [14]:
pip install ydata-profiling

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import ydata_profiling
from ydata_profiling import ProfileReport

profile = ProfileReport(df_csv)
profile.to_notebook_iframe()


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:00<00:00, 174.03it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Como duration_sec e num_frames são altamente correlacionadas (já que em todo vídeo um segundo tem 30 frames), podemos dar drop() em uma dessas colunas.

In [16]:
df_csv_dsdrop = df_csv.drop("duration_sec", axis=1)

In [17]:
dupes = df_csv_dsdrop.duplicated() # conferindo duplicatas sem a coluna "duration_sec"
sum(dupes)

0

In [18]:
df_csv_dsdrop.shape

(2501, 6)

# 2. JSON -> dataframe


Dado que o csv é a tabela principal, e cada instância dessa tabela tem um arquivo json correspondente, decidi expandir os dados em uma estrutura tabular. Dessa forma, cada frame pode ser ligado ao identificador do sinal do csv.

Começando por transformar os arquivos json em um dataframe:

In [179]:
'''import json
import os

# caminho_json = "/content/drive/My Drive/content/prova2/Sinais"
caminho_json = "Sinais"

json_records = []

for fname in os.listdir(caminho_json):
  if fname.endswith(".json"):
    file_path = os.path.join(caminho_json, fname)
    with open(file_path, "r", encoding="utf-8") as f:
      json_data = json.load(f) # Renamed variable here

    for frame_data in json_data["frames"]:
      frame_num = frame_data["frame"]
      for kp in frame_data["keypoints"]:
        json_records.append({
            "file_name": fname,
            "frame": frame_num,
            "id": kp["id"],
            "x": kp["x"],
            "y": kp["y"],
            "z": kp["z"],
            "visibility": kp["visibility"]
        })

# criar dataframe com todos os json
df_json = pd.DataFrame(json_records)
'''

'import json\nimport os\n\n# caminho_json = "/content/drive/My Drive/content/prova2/Sinais"\ncaminho_json = "Sinais"\n\njson_records = []\n\nfor fname in os.listdir(caminho_json):\n  if fname.endswith(".json"):\n    file_path = os.path.join(caminho_json, fname)\n    with open(file_path, "r", encoding="utf-8") as f:\n      json_data = json.load(f) # Renamed variable here\n\n    for frame_data in json_data["frames"]:\n      frame_num = frame_data["frame"]\n      for kp in frame_data["keypoints"]:\n        json_records.append({\n            "file_name": fname,\n            "frame": frame_num,\n            "id": kp["id"],\n            "x": kp["x"],\n            "y": kp["y"],\n            "z": kp["z"],\n            "visibility": kp["visibility"]\n        })\n\n# criar dataframe com todos os json\ndf_json = pd.DataFrame(json_records)\n'

In [19]:
import os
import json
import csv
import pandas as pd
import unicodedata

# ---------- CONFIG ----------
caminho_json = "Sinais"                # pasta com os JSONs
output_csv = "processed/df_json_cleaned.csv"
EPSILON = 0.4                         # considerar invisível se <= 0.4
THRESH = 0.5                           # remover se >= 50% invisível
# -----------------------------

# --- prepara arquivo de saída (remove se existir) ---
meta_map = {}
os.makedirs(os.path.dirname(output_csv) or ".", exist_ok=True)
if os.path.exists(output_csv):
    os.remove(output_csv)

header = ["file_name", "frame", "id", "x", "y", "z", "visibility"]

with open(output_csv, "w", newline="", encoding="utf-8") as out_f:
    writer = csv.writer(out_f)
    writer.writerow(header)

    files = sorted(os.listdir(caminho_json))
    for idx, fname in enumerate(files, start=1):
        if not fname.lower().endswith(".json"):
            continue

        file_path = os.path.join(caminho_json, fname)
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                json_data = json.load(f)
        except Exception as e:
            print(f"Erro lendo {fname}: {e}")
            continue

        frames = json_data.get("frames", [])
        # 1ª passagem: contar total e invisível por id
        total_counts = {}
        invis_counts = {}
        for fr in frames:
            for kp in fr.get("keypoints", []):
                kid = kp["id"]
                total_counts[kid] = total_counts.get(kid, 0) + 1
                if kp.get("visibility", 0) <= EPSILON:
                    invis_counts[kid] = invis_counts.get(kid, 0) + 1

        # ids a remover neste arquivo
        ids_to_drop = {kid for kid, tot in total_counts.items()
                       if (invis_counts.get(kid, 0) / tot) >= THRESH}

        # 2ª passagem: escrever apenas keypoints que NÃO estão em ids_to_drop
        for fr in frames:
            fnum = fr.get("frame")
            for kp in fr.get("keypoints", []):
                kid = kp["id"]
                if kid in ids_to_drop:
                    continue
                x = kp.get("x")
                y = kp.get("y")
                z = kp.get("z")
                vis = kp.get("visibility")
                writer.writerow([fname, fnum, kid, x, y, z, vis])

        # feedback simples de progresso
        if idx % 100 == 0:
            print(f"Processados {idx} arquivos...")

print("Processamento concluído. Resultado em:", output_csv)


Processados 100 arquivos...
Processados 200 arquivos...
Processados 300 arquivos...
Processados 400 arquivos...
Processados 500 arquivos...
Processados 600 arquivos...
Processados 700 arquivos...
Processados 800 arquivos...
Processados 900 arquivos...
Processados 1000 arquivos...
Processados 1100 arquivos...
Processados 1200 arquivos...
Processados 1300 arquivos...
Processados 1400 arquivos...
Processados 1500 arquivos...
Processados 1600 arquivos...
Processados 1700 arquivos...
Processados 1800 arquivos...
Processados 1900 arquivos...
Processados 2000 arquivos...
Processados 2100 arquivos...
Processados 2200 arquivos...
Processados 2300 arquivos...
Processados 2400 arquivos...
Processados 2500 arquivos...
Processamento concluído. Resultado em: processed/df_json_cleaned.csv


In [20]:
df_json = pd.read_csv("processed/df_json_cleaned.csv")

In [21]:
df_json.head()

,file_name,frame,id,x,y,z,visibility
0,Adição_AP_1.json,0,0,438,184,-1.44,1.0
1,Adição_AP_1.json,0,1,462,154,-1.36,1.0
2,Adição_AP_1.json,0,2,474,154,-1.36,1.0
3,Adição_AP_1.json,0,3,484,154,-1.36,1.0
4,Adição_AP_1.json,0,4,421,155,-1.36,1.0


In [22]:
df_json.duplicated().sum()

np.int64(0)

Esse dataframe também possui acentos. Vamos removê-los da mesma forma do df_csv:

In [24]:
# normalizando a coluna "file_name"
df_json["file_name"] = df_json["file_name"].apply(remove_acentos)

print(df_json.head())

          file_name  frame  id    x    y     z  visibility
0  Adicao_AP_1.json      0   0  438  184 -1.44         1.0
1  Adicao_AP_1.json      0   1  462  154 -1.36         1.0
2  Adicao_AP_1.json      0   2  474  154 -1.36         1.0
3  Adicao_AP_1.json      0   3  484  154 -1.36         1.0
4  Adicao_AP_1.json      0   4  421  155 -1.36         1.0


In [25]:
df_json["file_name"].value_counts()

file_name
Colega_TS_10.json       8528
Ensinar_TS_10.json      7566
Coesao_CS_10.json       7202
Estudar_TS_10.json      6838
Curso_TS_10.json        6682
                        ... 
Coerencia_MA_8.json      825
Coerencia_MA_10.json     825
Colega_MA_10.json        800
Filosofia_LS_8.json      759
Historia_EL_3.json       567
Name: count, Length: 2501, dtype: int64

In [26]:
df_json.shape

(7346183, 7)

## Merge JSON + CSV

Merge inner join entre o csv e o novo dataframe dos jsons pelo file_name

In [27]:
df_final = pd.merge(df_json, df_csv_dsdrop, on="file_name", how="inner")

In [28]:
# transformar em arquivo csv
df_final.to_csv("processed/df_final.csv", index=False)

In [29]:
df_final.shape

(7346183, 12)

In [30]:
df_final.duplicated().sum()

np.int64(0)

In [31]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7346183 entries, 0 to 7346182
Data columns (total 12 columns):
 #   Column      Dtype  
---  ------      -----  
 0   file_name   object 
 1   frame       int64  
 2   id          int64  
 3   x           int64  
 4   y           int64  
 5   z           float64
 6   visibility  float64
 7   width       int64  
 8   height      int64  
 9   num_frames  int64  
 10  sinal       object 
 11  interprete  object 
dtypes: float64(2), int64(7), object(3)
memory usage: 672.6+ MB


In [52]:
# normalizar x e y em função da largura e altura
df_final["x"] = df_final["x"] / df_final["width"]
df_final["y"] = df_final["y"] / df_final["height"]

In [57]:
# drop em width e height
df_final = df_final.drop(["width", "height"], axis=1)

In [204]:
df_final.drop(["z"], axis=1, inplace=True)

A fim de diminuir a quantidade de linhas, vamos remover aquelas cujo id é invisível ou quase isso em pelo menos 50% do arquivo

In [192]:
'''# 1. Calcular fração de visibilidade zero por (file_name, id)
frac_invisivel = (
    df_final.assign(invisivel = df_final["visibility"] <= 0.4)
    .groupby(["file_name", "id"])["invisivel"]
    .mean()
    .reset_index(name="frac_invisivel")
)

# 2. Marcar quais pares devem ser removidos
pares_ruins = frac_invisivel.query("frac_invisivel >= 0.5")[["file_name", "id"]]

# 3. Fazer merge para marcar no dataframe original
df_merged = df_final.merge(pares_ruins, on=["file_name","id"], how="left", indicator=True)

# 4. Manter apenas os que NÃO estão na lista de ruins
df_limpo = df_merged[df_merged["_merge"] == "left_only"]'''

'# 1. Calcular fração de visibilidade zero por (file_name, id)\nfrac_invisivel = (\n    df_final.assign(invisivel = df_final["visibility"] <= 0.4)\n    .groupby(["file_name", "id"])["invisivel"]\n    .mean()\n    .reset_index(name="frac_invisivel")\n)\n\n# 2. Marcar quais pares devem ser removidos\npares_ruins = frac_invisivel.query("frac_invisivel >= 0.5")[["file_name", "id"]]\n\n# 3. Fazer merge para marcar no dataframe original\ndf_merged = df_final.merge(pares_ruins, on=["file_name","id"], how="left", indicator=True)\n\n# 4. Manter apenas os que NÃO estão na lista de ruins\ndf_limpo = df_merged[df_merged["_merge"] == "left_only"]'

In [117]:
import pandas as pd

# filtrar apenas keypoints de 11 a 24
df_kp = df_final[df_final["id"].between(11, 24)]

# calcular mean, std e var de x e y por file_name e id
agg = (df_kp.groupby(["file_name", "id"])
              .agg(x_mean=("x", "mean"),
                   x_std=("x", "std"),
                   x_var=("x", "var"),
                   y_mean=("y", "mean"),
                   y_std=("y", "std"),
                   y_var=("y", "var"))
              .reset_index())

# transformar em colunas (wide format)
agg_wide = agg.pivot(index="file_name", columns="id")
agg_wide.columns = [f"kp_{id}_{stat}" for stat,id in agg_wide.columns]
agg_wide = agg_wide.reset_index()

# pegar apenas uma linha por file_name dos metadados (sinal, interprete etc.)
meta = df_final.groupby("file_name").agg({
    "sinal": "first",
    "interprete": "first"
}).reset_index()

# merge final: metadados + features agregadas
video_df = meta.merge(agg_wide, on="file_name", how="left")

In [118]:
video_df.shape

(2501, 87)

## Redução de dimensionalidade

Redução de dimensionalidade via correlação de Pearson vai ajudar a eliminar as colunas altamente correlacionadas (que tendem a carregar informações redundantes), buscando manter apenas uma delas. De início já é possível notar que "num_frames" e "duration_sec" são exemplo de um caso. As colunas x e y em comparação à width e height também.

In [119]:
video_df = pd.DataFrame(video_df)

In [120]:
video_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2501 entries, 0 to 2500
Data columns (total 87 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   file_name     2501 non-null   object 
 1   sinal         2501 non-null   object 
 2   interprete    2501 non-null   object 
 3   kp_11_x_mean  2501 non-null   float64
 4   kp_12_x_mean  2501 non-null   float64
 5   kp_13_x_mean  2501 non-null   float64
 6   kp_14_x_mean  2501 non-null   float64
 7   kp_15_x_mean  2501 non-null   float64
 8   kp_16_x_mean  2501 non-null   float64
 9   kp_17_x_mean  2498 non-null   float64
 10  kp_18_x_mean  2501 non-null   float64
 11  kp_19_x_mean  2499 non-null   float64
 12  kp_20_x_mean  2501 non-null   float64
 13  kp_21_x_mean  2501 non-null   float64
 14  kp_22_x_mean  2501 non-null   float64
 15  kp_23_x_mean  2499 non-null   float64
 16  kp_24_x_mean  2501 non-null   float64
 17  kp_11_x_std   2501 non-null   float64
 18  kp_12_x_std   2501 non-null 

A coluna z também não será muito útil visto que é a distância do intérprete para a câmera

In [ ]:
# verificando a coluna alvo (sinal)
video_df["sinal"].value_counts()

sinal
Adição              101
Aluno               101
Ângulo              100
Antropologia        100
Apontador           100
Apostila            100
Biologia            100
Capítulo            100
Classe              100
Disciplina          100
Coerência           100
Coesão              100
Colega              100
Conceito            100
Contexto            100
Curso               100
Dicionário          100
Filosofia           100
Ensinar             100
Escola              100
Estudar             100
Geografia           100
Física              100
História            100
Bolsa de Estudos     99
Name: count, dtype: int64

In [64]:
video_df["sinal"].nunique()

25

In [121]:
video_df.isnull().sum().sum()

np.int64(42)

In [122]:
video_df = video_df.fillna(video_df.mean(numeric_only=True))

In [123]:
video_df.isnull().sum().sum()

np.int64(0)

A coluna alvo tem 25 valores.

## Redução de dimensionalidade

Já removi algumas colunas que o profiling mostrou como sendo altamente correlacionadas. Porém, com o merge e outros fatores, agora tenho 87 colunas. Aplicar a técnica de análise de correlação de Pearson vai ajudar a eliminar mais colunas que carregam informações redundantes (manter apenas uma delas)

In [124]:
# separar os dados e a variável alvo (rótulo)
dados_corr = video_df.drop(columns=["sinal"], axis=1)
# também vou dar drop em file_name e interprete por serem variáveis nominais
dados_corr = dados_corr.drop(columns=["file_name", "interprete"], axis=1)

rotulo_corr = video_df["sinal"]

In [125]:
# calcular matriz de correlação de pearson
# usa .abs() pois pega correlações positivas e negativas
correlation_matrix = dados_corr.corr(method="pearson").abs()

In [126]:
# atualmente temos 87 colunas
# então é esperado ficar com 84 após a remoção da coluna alvo + file_name e interprete
dados_corr.shape

(2501, 84)

In [127]:
# selecionar apenas a parte superior da matriz de correlação (triângulo superior)
# cria-se uma matriz de booleanos, tipo uma máscara, em que o triângulo superior é todo preenchido com True
# e a diagonal principal e o triângulo inferior com False;
# Depois utiliza essa máscara para selecionar apenas o triângulo superior
triangulo_superior = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
triangulo_superior.shape

(84, 84)

Assim é possível identificar quais colunas têm alta correlação. Quando colunas tiverem correlação maior que 90%, serão removidas.

In [128]:
# encontrar colunas com correlação maior que 0.90
limite_correlacao = 0.90
to_drop = [column for column in triangulo_superior.columns if any(triangulo_superior[column] > limite_correlacao)]

# remover essas colunas
dados_corr_filtered = dados_corr.drop(columns=to_drop)

print(f"{len(to_drop)} colunas para remover por alta correlação.")

56 colunas para remover por alta correlação.


In [129]:
# novo datarame sem essas colunas
df_corr_filtered = pd.concat([dados_corr_filtered, rotulo_corr], axis=1)

df_corr_filtered.shape

(2501, 29)

Agora, redução de dimensionalidade com PCA

In [130]:
# Separar novamente os dados da variável alvo (rótulo)
dados_pca = df_corr_filtered.drop(columns=["sinal"])
rotulo_pca = df_corr_filtered["sinal"]

Como PCA é sensível à escala das variáveis, se um atributo tiver valores muito grandes e outro atributo tiver valores muito pequenos, a variânvia do primeiro vai dominar a análise. Pra isso, vou aplicar standardScaler() pra normalizar. Essa função transforma os dados para que cada feature tenha média 0 e desvio padrão 1, o que garante que todas as variáveis tenham o mesmo peso na hora de calcular os componentes principais

In [131]:
# normalizando os dados
scaler = StandardScaler()
dados_normalizados = scaler.fit_transform(dados_pca)

Aqui, iremos escolher preservar 95% da variância total dos dados originais, assim queremos manter a quantidade mínima de componentes nescessários para poder preservar essa variância;

E com o método fit_transform, calcularemos os componentes principais com base na variância dos dados, isso projeta os dados originais no novo sistema de coordenadas reduzido

In [132]:
# aplicando o PCA mantendo 95% da variância
pca = PCA(n_components=0.95)
dados_pca_transformado = pca.fit_transform(dados_normalizados)

# vendo quantos componentes foram mantidos
print(f"{pca.n_components_} componentes mantidos.")

17 componentes mantidos.


# 4. Treino e comparação de desempenho

In [ ]:
# validação cruzada estratificada:

folds = 10 # 10 divisões
r_state = 42 # fixando random_state

# separa features, rótulos e grupos
X = dados_pca_transformado
y = rotulo_pca
groups = video_df["interprete"]  # mesma ordem das instâncias

# validador estratificado por sinal, mas separado por intérprete
# shuffle=True pra embaralhar os dados antes de dividir
validador = StratifiedGroupKFold(n_splits=folds, shuffle=True, random_state=r_state)

F1 médio - Árvore de Decisão: 0.17867848599885047
F1 médio - MLP: 0.3634849729459898


In [ ]:
# modelos
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=r_state)
modelo_mlp = MLPClassifier(max_iter=1000, random_state=r_state)
modelo_knn = KNeighborsClassifier(n_neighbors=5)

# cross_val_score com grupos
f1_arvore = cross_val_score(modelo_rf, X, y, groups=groups, cv=validador, scoring="f1_macro")
f1_mlp = cross_val_score(modelo_mlp, X, y, groups=groups, cv=validador, scoring="f1_macro")
f1_knn = cross_val_score(modelo_knn, X, y, groups=groups, cv=validador, scoring="f1_macro")

print("F1 médio - Árvore de Decisão:", f1_arvore.mean())
print("F1 médio - MLP:", f1_mlp.mean())
print("F1 médio - KNN:", f1_knn.mean())

F1 médio - Árvore de Decisão: 0.32771219900021237
F1 médio - MLP: 0.3634849729459898
F1 médio - KNN: 0.28355973099876924


MLP teve o melhor desempenho, o que faz sentido já que os dados foram transformados por PCA e redes neurais conseguem captar as relações não-lineares que o PCA mantém.

Matrizes de confusão:

In [ ]:
# Lista de modelos e nomes para iteração
modelos = [
    ("Random Forest", modelo_rf),
    ("MLP", modelo_mlp),
    ("KNN", modelo_knn)
]

# Para cada modelo, gera a matriz de confusão e plota
for nome, modelo in modelos:
    # cross_val_predict retorna as previsões usando validação cruzada
    y_pred = cross_val_predict(modelo, X, y, groups=groups, cv=validador)
    
    # matriz de confusão
    cm = confusion_matrix(y, y_pred, labels=np.unique(y))
    
    # plotando a matriz
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))
    disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
    plt.title(f"Matriz de Confusão - {nome}")
    plt.show()

C:\Users\Win\AppData\Local\Temp\ipykernel_20540\3510591458.py:24: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\Win\AppData\Local\Temp\ipykernel_20540\3510591458.py:24: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\Win\AppData\Local\Temp\ipykernel_20540\3510591458.py:24: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
